In [25]:
import pandas as pd
import numpy as np
import time 
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
base_link = "https://www.boligsiden.dk/tilsalg/{h_type}?sortAscending=true&page={p_number}"


In [58]:
house_types = ['villa','ejerlejlighed','raekkehus','andelsbolig','villalejlighed','landejendom']
p_numbers = range(1, 101, 1)

def get_links(driver):
    links = driver.find_elements(By.XPATH, "//a[starts-with(@href, '/adresse/')]")  # Used find_elements instead of find_element
    list_of_links = [link.get_attribute('href') for link in links]
    return list_of_links

def get_div_links(driver):
    div_links = driver.find_elements(By.XPATH, "//h3[@id]//a")  # Locate <a> tags inside <h3> tags with an id attribute
    list_of_div_links = [link.get_attribute('href') for link in div_links]
    return list_of_div_links

def selenium_scroller(driver, search_url):
    url = search_url
    driver.get(url)
    scroll_height = 0
    last_height = driver.execute_script("return document.body.scrollHeight")
    time.sleep(3)
    property_listings_div = driver.find_element(By.CSS_SELECTOR, '#__next > div.min-h-screen.flex.flex-col > main > div > div.flex-1.flex.flex-row.items-stretch.xl\:py-10.xl\:container.xl\:mx-auto > div.w-full.xl\:w-9\/12.xl\:pr-2 > div.flex-1.h-full.flex.flex-col.space-y-10.mb-4.xl\:mb-0 > div.mb-10.space-y-4.flex.flex-col.xl\:px-0')
    while True:
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", property_listings_div)
        time.sleep(1)
        new_scroll_height = driver.execute_script("return arguments[0].scrollHeight;", property_listings_div)
        if new_scroll_height == scroll_height:
            break
        scroll_height = new_scroll_height
    list_of_links = get_links(driver)
    list_of_div_links = get_div_links(driver)
    print(len(list_of_links), len(list_of_div_links))
    return list_of_links, list_of_div_links

all_links = []
all_div_links = []

driver = webdriver.Chrome()
for h_type in house_types:
    for p_number in p_numbers:
        BASE_URL = f"https://www.boligsiden.dk/tilsalg/{h_type}?sortAscending=true&page={p_number}"
        links, div_links = selenium_scroller(driver, BASE_URL)
        all_links.extend(links)
        all_div_links.extend(div_links)
driver.quit()
def remove_duplicates(input_list):
    return list(dict.fromkeys(input_list))

all_links = remove_duplicates(all_links)
for links in all_links:
    print(links)

with open('all_links.txt', 'w') as f:
    for link in all_links:
        f.write(link + '\n')

# Save all_div_links to a file
with open('all_div_links.txt', 'w') as f:
    for link in all_div_links:
        f.write(link + '\n')

115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 0
115 

In [55]:
info = ['price', 'road','city', 'energy', 'building type', 'udbetaling','area', 'land area','rooms','ejerudg','constructed','latest remodel']#,'levels','toilets','heating']
count =0
elements = []
for link in all_links:
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    
    price = soup.select("#oversigt > div.flex.flex-col.divide-y.divide-gray-100.md\:flex-row.md\:justify-between.md\:divide-y-0 > div.py-5.md\:pl-0.px-2.md\:px-6.flex.flex-col > div:nth-child(1) > div.cursor-pointer > div > div > h2")
    price_text = price[0].text if price else None
    
    road = soup.select("#oversigt > div.bg-lightgray > div > div.transition-\[height\].ease-in-out.duration-700.hidden.absolute.md\:flex.bottom-0.inset-x-0.z-20.text-white.justify-between.items-end.h-\[72px\] > div > div.h-full.flex.p-4.px-5.gap-8 > h1 > span.text-lg.font-semibold")
    road_text = road[0].text if road else None

    city = soup.select("#oversigt > div.bg-lightgray > div > div.transition-\[height\].ease-in-out.duration-700.hidden.absolute.md\:flex.bottom-0.inset-x-0.z-20.text-white.justify-between.items-end.h-\[72px\] > div > div.h-full.flex.p-4.px-5.gap-8 > h1 > span.text-xs.font-light")
    city_text = city[0].text if city else None
    
    energy = soup.select("#Lag_1 > title")
    energy_text = energy[0].text if energy else None
    
    building_type = soup.select("#oversigt > div.bg-lightgray > div > div.transition-\[height\].ease-in-out.duration-700.hidden.absolute.md\:flex.bottom-0.inset-x-0.z-20.text-white.justify-between.items-end.h-\[72px\] > div > div.h-full.flex.p-4.px-5.gap-8 > div > p")
    building_type_text = building_type[0].text if building_type else None
    
    udbetaling = soup.select("#oversigt > div.flex.flex-col.divide-y.divide-gray-100.md\:flex-row.md\:justify-between.md\:divide-y-0 > div.py-5.md\:pl-0.px-2.md\:px-6.flex.flex-col > div.text-gray-600.text-sm.md\:mb-2")
    udbetaling_text = udbetaling[0].text if udbetaling else None
    area = soup.select("#oversigt > div.flex.flex-col.divide-y.divide-gray-100.md\:flex-row.md\:justify-between.md\:divide-y-0 > div.flex.flex-col.md\:flex-row.justify-end > div > div:nth-child(1) > div > div.w-full.flex.justify-start > div > div > span")
    area_text = area[0].text if area else None
    land_area = soup.select("#oversigt > div.flex.flex-col.divide-y.divide-gray-100.md\:flex-row.md\:justify-between.md\:divide-y-0 > div.flex.flex-col.md\:flex-row.justify-end > div > div:nth-child(2) > div > div.w-full.flex.justify-start > div > div > span")
    land_area_text = land_area[0].text if land_area else None
    rooms = soup.select("#oversigt > div.flex.flex-col.divide-y.divide-gray-100.md\:flex-row.md\:justify-between.md\:divide-y-0 > div.flex.flex-col.md\:flex-row.justify-end > div > div:nth-child(3) > div > span")
    rooms_text = rooms[0].text if rooms else None
    ejerudg = soup.select("#oversigt > div.flex.flex-col.divide-y.divide-gray-100.md\:flex-row.md\:justify-between.md\:divide-y-0 > div.flex.flex-col.md\:flex-row.justify-end > div > div:nth-child(4) > div > div.w-full.flex.justify-start > div > div > span")
    ejerudg_text = ejerudg[0].text if ejerudg else None
    constructed = soup.select("#oversigt > div.flex.flex-col.divide-y.divide-gray-100.md\:flex-row.md\:justify-between.md\:divide-y-0 > div.flex.flex-col.md\:flex-row.justify-end > div > div:nth-child(5) > div > span")
    constructed_text = constructed[0].text if constructed else None
    latest_remodel = soup.select("#oversigt > div.pb-5.px-6.text-sm > div:nth-child(1) > div:nth-child(2)")
    latest_remodel_text = latest_remodel[0].text if latest_remodel else None
    
    # levels = soup.select("")
    # _text = road[0].text if road else None
    # toilets = soup.select("")
    # _text = road[0].text if road else None
    # heating = soup.select("")
    # _text = road[0].text if road else None
    element = (price_text, road_text, city_text, energy_text,building_type_text,udbetaling_text,area_text,land_area_text,rooms_text,ejerudg_text, constructed_text,latest_remodel_text)
    elements.append(element)
    count += 1
    print(count, element)



1 ('4.995.000 kr.', 'Løvvænget 7', '3100 Hornbæk', 'Energimærke B', 'Villa', 'Udbetaling 250.000 kr.', '182 m²', 'Grund: 1157 m²', '5 værelser', 'Ejerudg.: 5.224 kr/md', 'Opført 1985', None)
2 ('1.295.000 kr.', 'Langemosevej 16', 'Langemose, 5610 Assens', 'Energimærke C', 'Villa', 'Udbetaling 65.000 kr.', '160 m²', 'Grund: 1580 m²', '6 værelser', 'Ejerudg.: 1.461 kr/md', 'Opført 1877', None)
3 ('5.295.000 kr.', 'Haldsøvej 22', '8240 Risskov', 'Energimærke C', 'Villa', 'Udbetaling 25.000 kr.', '181 m²', 'Grund: 1000 m²', '6 værelser', 'Ejerudg.: 3.833 kr/md', 'Opført 1999', None)
4 ('3.295.000 kr.', 'Julagervænget 105', 'Hjallese, 5260 Odense S', 'Energimærke D', 'Villa', 'Udbetaling 165.000 kr.', '150 m²', 'Grund: 765 m²', '4 værelser', 'Ejerudg.: 2.441 kr/md', 'Opført 1973', None)
5 ('2.250.000 kr.', 'Højvangsvej 49', '9400 Nørresundby', 'Energimærke F', 'Villa', 'Udbetaling 115.000 kr.', '95 m²', 'Grund: 569 m²', '5 værelser', 'Ejerudg.: 2.332 kr/md', 'Opført 1929', None)
6 ('2.295.0

In [56]:
df = pd.DataFrame(elements, columns=[info])
df

,price,road,city,energy,building type,udbetaling,area,land area,rooms,ejerudg,constructed,latest remodel
0,4.995.000 kr.,Løvvænget 7,3100 Hornbæk,Energimærke B,Villa,Udbetaling 250.000 kr.,182 m²,Grund: 1157 m²,5 værelser,Ejerudg.: 5.224 kr/md,Opført 1985,None
1,1.295.000 kr.,Langemosevej 16,"Langemose, 5610 Assens",Energimærke C,Villa,Udbetaling 65.000 kr.,160 m²,Grund: 1580 m²,6 værelser,Ejerudg.: 1.461 kr/md,Opført 1877,None
2,5.295.000 kr.,Haldsøvej 22,8240 Risskov,Energimærke C,Villa,Udbetaling 25.000 kr.,181 m²,Grund: 1000 m²,6 værelser,Ejerudg.: 3.833 kr/md,Opført 1999,None
3,3.295.000 kr.,Julagervænget 105,"Hjallese, 5260 Odense S",Energimærke D,Villa,Udbetaling 165.000 kr.,150 m²,Grund: 765 m²,4 værelser,Ejerudg.: 2.441 kr/md,Opført 1973,None
4,2.250.000 kr.,Højvangsvej 49,9400 Nørresundby,Energimærke F,Villa,Udbetaling 115.000 kr.,95 m²,Grund: 569 m²,5 værelser,Ejerudg.: 2.332 kr/md,Opført 1929,None
5,2.295.000 kr.,Stolpekærvej 16,3370 Melby,Energimærke D,Villa,Udbetaling 115.000 kr.,130 m²,Grund: 886 m²,6 værelser,Ejerudg.: 3.094 kr/md,Opført 1965,None
6,13.500.000 kr.,Sanskevej 11,2960 Rungsted Kyst,Energimærke C,Villa,Udbetaling 675.000 kr.,254 m²,Grund: 982 m²,7 værelser,Ejerudg.: 8.295 kr/md,Opført 1917,None
7,1.345.000 kr.,Kildevej 4,4771 Kalvehave,Energimærke D,Villa,Udbetaling 70.000 kr.,120 m²,Grund: 823 m²,3 værelser,Ejerudg.: 2.015 kr/md,Opført 1973,None
8,2.995.000 kr.,Thorsvej 35,7000 Fredericia,Energimærke C,Villa,Udbetaling 150.000 kr.,138 m²,Grund: 457 m²,5 værelser,Ejerudg.: 2.210 kr/md,Opført 1925,None
9,1.145.000 kr.,Møllevej 8,7830 Vinderup,Energimærke D,Villa,Udbetaling 60.000 kr.,127 m²,Grund: 663 m²,5 værelser,Ejerudg.: 1.551 kr/md,Opført 1914,None


In [39]:
for links in all_links:
    print(links)

https://www.boligsiden.dk/adresse/frydenlund-176-7120-vejle-oest-06302986_176_______?udbud=b5c1ded1-2f9e-4c89-beef-d3cd52dd57dd
https://www.boligsiden.dk/adresse/frydenlund-176-7120-vejle-oest-06302986_176_______?udbud=b5c1ded1-2f9e-4c89-beef-d3cd52dd57dd
https://www.boligsiden.dk/adresse/astersvej-10-3310-oelsted-02600062__10_______?udbud=11dc8954-0275-4099-b1ca-d8a43db4b545
https://www.boligsiden.dk/adresse/astersvej-10-3310-oelsted-02600062__10_______?udbud=11dc8954-0275-4099-b1ca-d8a43db4b545
https://www.boligsiden.dk/adresse/borup-alle-40-8920-randers-nv-07300208__40_______?udbud=2c9378f5-640c-442f-9ea6-f2e8f13539d7
https://www.boligsiden.dk/adresse/borup-alle-40-8920-randers-nv-07300208__40_______?udbud=2c9378f5-640c-442f-9ea6-f2e8f13539d7
https://www.boligsiden.dk/adresse/padkaervej-91-9670-loegstoer-08200994__91_______?udbud=d80d15e5-4a7e-4ed9-9a08-4684a7e49837
https://www.boligsiden.dk/adresse/padkaervej-91-9670-loegstoer-08200994__91_______?udbud=d80d15e5-4a7e-4ed9-9a08-4684a